X-Grad-Cam: Instead of global average pooling, XGrad-CAM normalizes the gradients using a term that adjusts for feature importance per location.
This normalization helps refine the heatmap, making it sharper and more localized.
More accurate feature weighting than Grad-CAM.
Slightly more computationally expensive than standard Grad-CAM.

In [ ]:
import numpy as np
from pytorch_grad_cam.base_cam import BaseCAM


class XGradCAM(BaseCAM):
    def __init__(
            self,
            model,
            target_layers,
            reshape_transform=None):
        super(
            XGradCAM,
            self).__init__(
            model,
            target_layers,
            reshape_transform)

    def get_cam_weights(self,
                        input_tensor,
                        target_layer,
                        target_category,
                        activations,
                        grads):
        sum_activations = np.sum(activations, axis=(2, 3))
        eps = 1e-7
        weights = grads * activations / \
            (sum_activations[:, :, None, None] + eps)
        weights = weights.sum(axis=(2, 3))
        return weights

Chat GPT 

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import cv2
import matplotlib.pyplot as plt
from pytorch_grad_cam import XGradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image

# Load Xception Model (same as LIME notebook)
from training.detectors import DETECTOR
import yaml

def load_config(path, additional_args={}):
    with open(path, 'r') as f:
        config = yaml.safe_load(f)
    return config

path = "./training/config/detector/xception.yaml"
additional_args = {'test_batchSize': 12, 'pretrained': './weights/ckpt_best.pth'}
config = load_config(path, additional_args=additional_args)
model_class = DETECTOR[config['model_name']]
model = model_class(config)
model.eval()

# Select the last convolutional layer
TARGET_LAYER = model.backbone.features[-1]  # Adjust if needed

# Define function to apply XGrad-CAM
def apply_xgrad_cam(model, images, target_class=1):
    cam = XGradCAM(model=model, target_layers=[TARGET_LAYER])
    targets = [ClassifierOutputTarget(target_class)]
    grayscale_cam = cam(input_tensor=images, targets=targets)  # Generate CAM
    grayscale_cam = grayscale_cam[0, :]  # Extract CAM for first batch
    return grayscale_cam

# Function to overlay heatmap
def overlay_heatmap(image, cam):
    image = np.transpose(image.numpy(), (1, 2, 0))  # Convert from Tensor format
    image = (image - image.min()) / (image.max() - image.min())  # Normalize
    cam = cv2.resize(cam, (224, 224))  # Resize CAM to match image size
    heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
    overlay = (heatmap / 255) * 0.5 + image
    overlay = (overlay - overlay.min()) / (overlay.max() - overlay.min())
    return overlay